In [1]:
import flowcode
import processing
import res_flow_vis as visual

import torch
import numpy as np

In [2]:
#Filename associated with this specific run
filename = "testrun1"

In [3]:
#Initiate a processor to handle data
mpc = processing.Processor_cond(N_min=0)

In [4]:
#Load raw data
Data, N_stars, M_stars, M_dm = mpc.get_data("all_sims")

In [5]:
#Clean data
Data_const, N_stars_const, M_stars_const = mpc.constraindata(Data)

Cut out 0 of 95 galaxies, 4417968 of 34878379 stars (~13%).


In [6]:
#Choose device
device = "cuda"

In [7]:
#Hyperparameters of the flow
LAYER_TYPE = flowcode.NSF_CL2
N_LAYERS = 8
COND_INDS = np.array([10])
DIM_COND = COND_INDS.shape[0]
DIM_NOTCOND = Data_const[0].shape[1] - DIM_COND
SPLIT = 0.5
K = 8
B = 3
BASE_NETWORK = flowcode.MLP
BASE_NETWORK_N_LAYERS = 4
BASE_NETWORK_N_HIDDEN = 16
BASE_NETWORK_LEAKY_RELU_SLOPE = 0.2

SPLIT = {"split":SPLIT} if LAYER_TYPE == flowcode.NSF_CL else {}

In [8]:
#Instantiate the model
model = flowcode.NSFlow(N_LAYERS, DIM_NOTCOND, DIM_COND, LAYER_TYPE, **SPLIT, K=K, B=B, network=BASE_NETWORK, network_args=(BASE_NETWORK_N_HIDDEN,BASE_NETWORK_N_LAYERS,BASE_NETWORK_LEAKY_RELU_SLOPE))
model = model.to(device)

In [9]:
#Training hyperparameters
N_EPOCHS = 3
INIT_LR = 0.01
BATCH_SIZE = 1024

In [10]:
#Prepare data for flow
Data_flow = mpc.Data_to_flow(mpc.diststack(Data_const))

In [ ]:
#Save relevant data to the drive for external python file (device needs to be GPU) to do the training in background...
torch.save(Data_flow, "data_cond_trainer.pth")
torch.save(model, "model_cond_trainer.pth")
np.save("params_cond_trainer.npy", np.append(COND_INDS,np.array([N_EPOCHS,INIT_LR,BATCH_SIZE])))
np.save("filename_cond_trainer.npy", filename)

In [11]:
#...OR train here
import time
train_loss_saver = []
start = time.perf_counter()
flowcode.train_flow(model, Data_flow, COND_INDS, N_EPOCHS, lr=INIT_LR, batch_size=BATCH_SIZE, loss_saver=train_loss_saver)
end = time.perf_counter()
torch.save(model.state_dict(), f"{filename}.pth")
np.save(f"loss_{filename}.npy",np.array(train_loss_saver+[end-start]))

Step 0 of 89240, Loss:15.09046745300293, lr=0.01
Step 100 of 89240, Loss:12.15351354598999, lr=0.009321643399992438
Step 200 of 89240, Loss:8.996059532165527, lr=0.008689303567662258
Step 300 of 89240, Loss:7.996769876480102, lr=0.008099858925202963
Step 400 of 89240, Loss:8.193581714630128, lr=0.007550399649098802
Step 500 of 89240, Loss:7.52669587135315, lr=0.007038213305632705
Step 600 of 89240, Loss:7.526951999664306, lr=0.006560771460819006
Step 700 of 89240, Loss:7.291182794570923, lr=0.0061157171986602225
Step 800 of 89240, Loss:7.254281368255615, lr=0.005700853486111131
Step 900 of 89240, Loss:6.803214254379273, lr=0.0053141323273131705
Step 1000 of 89240, Loss:7.283186197280884, lr=0.004953644653558525
Step 1100 of 89240, Loss:6.582707529067993, lr=0.004617610899075165
Step 1200 of 89240, Loss:6.653057689666748, lr=0.004304372216109717
Step 1300 of 89240, Loss:6.575308990478516, lr=0.004012382285940997
Step 1400 of 89240, Loss:6.3289796733856205, lr=0.003740199685398845
Step 1

KeyboardInterrupt: 

In [ ]:
#Load in training results:
model.load_state_dict(torch.load(f"{filename}.pth"))
loss_results = np.load(f"loss_{filename}.npy")
loss_results, tot_time = loss_results[:-1], loss_results[-1]

In [ ]:
#Get a sample from the flow

#Set a condition for the sample
condition = mpc.diststack(Data_const)[:,COND_INDS]

#Get sample
flow_sample = mpc.galaxysplit(mpc.sample_to_Data(mpc.sample_Conditional(model, COND_INDS, condition)), N_stars_const)

In [1]:
### Visualize Data

In [ ]:
#Get multiple galaxy plot
visual.plot_conditional(flow_sample, M_stars_const, type="ofe", show="page", label=filename, scale="lin", gridsize=100, cmap="coolwarm")

In [ ]:
#Get comparison plot of single galaxy

visual.get_result_plots(Data_const[50], flow_sample[50], label=filename)